In [1]:
import numpy as np
import torch
from ucimlrepo import fetch_ucirepo 
import torch.nn as nn     
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import main
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
y["class"]=y["class"].replace("Iris-setosa",0.0)
y["class"]=y["class"].replace("Iris-versicolor",1.0)
y["class"]=y["class"].replace("Iris-virginica",2.0)
# metadata 
X=X.values
y=y.values
y=y.squeeze(axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
mean = X_train.mean(axis=0)  # Mean of training set
std = X_train.std(axis=0)  # Standard deviation of training set

# Avoid division by zero for constant features
std[std == 0] = 1

# Apply mean normalization (centering and scaling) to training and test sets
X_train_numpy = (X_train - mean) / std
X_test_numpy = (X_test - mean) / std

X_train=torch.FloatTensor(X_train_numpy)
X_test=torch.FloatTensor(X_test_numpy)
y_train=torch.LongTensor(y_train)
y_test=torch.LongTensor(y_test)

C:\Users\Sebas\AppData\Local\Temp\ipykernel_10812\167960093.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["class"]=y["class"].replace("Iris-setosa",0.0)
C:\Users\Sebas\AppData\Local\Temp\ipykernel_10812\167960093.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y["class"]=y["class"].replace("Iris-versicolor",1.0)
C:\Users\Sebas\AppData\Local\Temp\ipykernel_10812\167960093.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the

In [5]:
### Version one with hard thresholding
class Model(nn.Module):
    def __init__(self,in_features=4,h1=16,out=3):
        super().__init__()
        self.fc1=nn.Linear(in_features,h1,bias=False)
        self.out=nn.Linear(h1,out)
    def forward(self,x):
        hidden_activation = F.relu(self.fc1(x))  # Hidden layer with ReLU activation
        output = self.out(hidden_activation)  # Final output
        return output, hidden_activation
def iris_training(h1,alpha):
    model=Model(h1=h1)
    W_untrained=copy.deepcopy(next(model.parameters())).detach().numpy().astype(float)
    #print(W_untrained)
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
    epochs=100
    losses=[]
    for i in range(epochs):
        y_pred,hidden_activation=model.forward(X_train)
        classification_loss=loss=criterion(y_pred,y_train)
        neuron_activity_loss = -torch.mean(F.relu(hidden_activation))
        loss=classification_loss+alpha*neuron_activity_loss
        losses.append(classification_loss.detach().numpy())
        #if i % 10 == 0:
           # print(f'Epoch: {i} and loss: {loss}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    W_trained=next(model.parameters()).detach().numpy().astype(float)
    return W_untrained,W_trained,classification_loss.detach().numpy()
#plt.plot(range(epochs),losses)


In [8]:
for m in range(14,35):
    print(f"start {m}")
    losses=[]
    untrained=[]
    trained=[]
    for i in range(500):
        res=iris_training(m,0.5)
        losses.append(res[2])
        untrained.append(res[0])
        trained.append(res[1])
    losses=np.array(losses)
    print(np.mean(losses))
    res=0
    for W in untrained:
    # print(W.T.shape)
        if not main.findCell(np.array([]),W):
            res+=2
    res
    train=0
    for W in trained:
        if not main.findCell(np.array([]), W):
            train+=2
    print(f"m={m} untrained= {res},trained = {train}")


start 14
0.18699163
m=14 untrained= 10,trained = 6
start 15
0.1840094
m=15 untrained= 38,trained = 44
start 16
0.18240489
m=16 untrained= 106,trained = 108
start 17
0.17959549
m=17 untrained= 176,trained = 188
start 18
0.17863674
m=18 untrained= 332,trained = 334
start 19
0.17853501
m=19 untrained= 472,trained = 410
start 20
0.17706366
m=20 untrained= 566,trained = 540
start 21
0.17499143
m=21 untrained= 700,trained = 636
start 22
0.17461213
m=22 untrained= 728,trained = 688
start 23
0.17321113
m=23 untrained= 818,trained = 766
start 24
0.17255086
m=24 untrained= 882,trained = 842
start 25
0.17247441
m=25 untrained= 926,trained = 904
start 26
0.17130975
m=26 untrained= 942,trained = 950
start 27
0.17072603
m=27 untrained= 946,trained = 904
start 28
0.17038335
m=28 untrained= 978,trained = 956
start 29
0.16944845
m=29 untrained= 976,trained = 962
start 30
0.16991886
m=30 untrained= 994,trained = 970
start 31
0.16857992
m=31 untrained= 996,trained = 986
start 32
0.16825998
m=32 untrained